In [1]:
import openai
import os
import pandas as pd
import time
import sys
import scipy
import string
import contractions
import unicodedata
sys.path.insert(1,'../../src/')
import utils.general_tools as gt
import utils.setConfig as sc
config = pd.read_csv(f'../../bucket/config/{sc.select_data_folder()}',index_col=0)
import model_development.generate_embeddings as generate_embeddings

In [2]:
dataset_path=config.loc['data_postETL_and_featureEng'][0]
embeddings_without_ChatGPT_path=config.loc['embeddings_dataset_without_ChatGPT'][0]
ChatGPT_opinion_path=config.loc['ChatGPT_opinion'][0]
embeddings_with_ChatGPT_path=config.loc['embeddings_with_ChatGPT'][0]
embeddings_only_ChatGPT_path=config.loc['embeddings_only_ChatGPT_path'][0]
data_ETL_and_chatGPT_path=config.loc['data_ETL_and_chatGPT'][0]

In [3]:
dataset = pd.read_csv(dataset_path)
os.environ['bearer_token']=input('what is your account\'s bearer_token?')
bearer_token=os.environ.get('bearer_token')
openai.api_key= bearer_token

##### Generate Embedding v0 (without ChatGPT Opinion)

In [ ]:
# this code may not be updated, please use the python functions from src folder.
start_from_scratch=False#to be function variable
embeddings=pd.read_csv(embeddings_without_ChatGPT_path)
if start_from_scratch:
    embeddings=gt.make_dataframe_empty(embeddings)
for i in range(embeddings.shape[0],dataset.shape[0]):
    start = time.time()
    term = str(list(dataset.loc[i])) # giving the list with its brackets in string format found to make the most sense
    print(i, term)
    txt2vec=openai.Embedding.create(
        model="text-embedding-ada-002",
        input=term
    )
    emb= txt2vec['data'][0]['embedding']
    embeddings.loc[i]=emb
    end = time.time()
    time.sleep(0.5)
    if i%50==0:# essentially a checkpoint where it saves embeddings every 50 iterations 
        pd.DataFrame(embeddings).to_csv(\
            embeddings_without_ChatGPT_path)
pd.DataFrame(embeddings).to_csv(\
            embeddings_without_ChatGPT_path)

In [5]:
generate_embeddings.gen_emb(bearer_token,embeddings_without_ChatGPT_path)

100%|██████████| 1015/1015 [11:51<00:00,  1.43it/s]


##### Get ChatGPT's opinion on the tweets

In [ ]:
generate_embeddings.get_chatGPT_opinion(bearer_token,exact_search_term='iPhone 15')

##### Generate Embedding v1 (with ChatGPT Opinion)

In [ ]:
# this code may not be updated, please use the python functions from src folder.
start_from_scratch=True#to be function variable
chatGPT_opinions=pd.read_csv(ChatGPT_opinion_path,index_col=0)
if start_from_scratch:
    chatGPT_opinions=gt.make_dataframe_empty(chatGPT_opinions)
for i in range(chatGPT_opinions.shape[0],dataset.shape[0]):
    start = time.time()
    tweet=dataset.loc[i].text
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are an unbiased smartphone product analyst"},
            {"role": "user", "content": "Briefly describe whether product feedback is being\
              given about iphone 6 in the following tweet: {}".format(tweet)}
    ]
    )
    chatGPT_response=gt.make_txt_NLP_friendly(completion.choices[0].message.content)
    chatGPT_opinions.loc[i]=[tweet,chatGPT_response]
    print(i,[tweet,chatGPT_response])
    end = time.time()
    time.sleep(0.5)
    if i%50==0:
        pd.DataFrame(chatGPT_opinions).to_csv(\
            ChatGPT_opinion_path)
pd.DataFrame(chatGPT_opinions).to_csv(\
        ChatGPT_opinion_path)

In [6]:
generate_embeddings.combine_repo_with_chatGPT_opinion()

In [ ]:
# this code may not be updated, please use the python functions from src folder.
chatGPT_response_collection=pd.read_csv(ChatGPT_opinion_path,index_col=0)
new_columns=dataset.columns.to_list()+['chatGPT_opinion']
dataset_with_GPT_opinion=pd.concat([dataset,chatGPT_response_collection['chatGPT_opinion']],axis=1,ignore_index=True)
dataset_with_GPT_opinion.columns=new_columns
dataset_with_GPT_opinion.head()
embeddings_with_ChatGPT=pd.read_csv(embeddings_with_ChatGPT_path,index_col=0)
embeddings_with_ChatGPT=gt.make_dataframe_empty(embeddings_with_ChatGPT)
for i in range(dataset_with_GPT_opinion.shape[0]):
    start = time.time()
    term = str(list(dataset_with_GPT_opinion.loc[i])) # giving the list with its brackets in string format found to make the most sense
    # print(i, term)
    txt2vec=openai.Embedding.create(
        model="text-embedding-ada-002",
        input=term
    )
    emb= txt2vec['data'][0]['embedding']
    embeddings_with_ChatGPT.loc[i]=emb
    end = time.time()
    time.sleep(0.5)
    if i%50==0:
        pd.DataFrame(embeddings_with_ChatGPT).to_csv(\
            embeddings_with_ChatGPT_path)
pd.DataFrame(embeddings_with_ChatGPT).to_csv(\
            embeddings_with_ChatGPT_path)

In [8]:
generate_embeddings.gen_emb(bearer_token,embeddings_with_ChatGPT_path,
        dataset_path=data_ETL_and_chatGPT_path)

100%|██████████| 200/200 [02:17<00:00,  1.45it/s]


##### Generate Embedding v2 (only ChatGPT Opinion)

In [ ]:
# this code may not be updated, please use the python functions from src folder.
ChatGPT_opinion=pd.read_csv(ChatGPT_opinion_path,index_col=0)
embeddings_only_ChatGPT=pd.read_csv(embeddings_only_ChatGPT_path,index_col=0)
embeddings_only_ChatGPT=gt.make_dataframe_empty(embeddings_only_ChatGPT)
for i in range(embeddings_with_ChatGPT.shape[0]):
    start = time.time()
    term = str(dataset_with_GPT_opinion.loc[i].chatGPT_opinion) # giving the list with its brackets in string format found to make the most sense
    # print(i, term)
    txt2vec=openai.Embedding.create(
        model="text-embedding-ada-002",
        input=term
    )
    emb= txt2vec['data'][0]['embedding']
    embeddings_only_ChatGPT.loc[i]=emb
    end = time.time()
    time.sleep(0.1)
    if i%50==0:
        pd.DataFrame(embeddings_only_ChatGPT).to_csv(\
            embeddings_only_ChatGPT_path)
pd.DataFrame(embeddings_only_ChatGPT).to_csv(\
            embeddings_only_ChatGPT_path)

In [9]:
generate_embeddings.gen_emb(bearer_token,embeddings_only_ChatGPT_path,
        dataset_path=ChatGPT_opinion_path)

100%|██████████| 200/200 [02:27<00:00,  1.36it/s]
